In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

In [2]:
num_classes = 10 # Digits
num_features = 784 # 28*28

# Parameters for Train the Model
learning_rate = 0.01
training_steps = 1000 
batch_size = 256
display_step = 50

In [3]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [5]:
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [6]:
def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)

def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

def accuracy(y_pred, y_true):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.optimizers.SGD(learning_rate)

def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    gradients = g.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [7]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 1144.456055, accuracy: 0.703125
step: 100, loss: 84.144630, accuracy: 0.925781
step: 150, loss: 96.152420, accuracy: 0.878906
step: 200, loss: 112.013008, accuracy: 0.894531
step: 250, loss: 284.094635, accuracy: 0.808594
step: 300, loss: 101.271980, accuracy: 0.878906
step: 350, loss: 79.172226, accuracy: 0.917969
step: 400, loss: 96.272499, accuracy: 0.890625
step: 450, loss: 43.292046, accuracy: 0.957031
step: 500, loss: 147.700500, accuracy: 0.851562
step: 550, loss: 87.479416, accuracy: 0.914062
step: 600, loss: 66.650803, accuracy: 0.933594
step: 650, loss: 67.545670, accuracy: 0.933594
step: 700, loss: 55.077805, accuracy: 0.937500
step: 750, loss: 99.826836, accuracy: 0.894531
step: 800, loss: 88.446930, accuracy: 0.910156
step: 850, loss: 60.933506, accuracy: 0.941406
step: 900, loss: 91.890671, accuracy: 0.886719
step: 950, loss: 45.065918, accuracy: 0.964844
step: 1000, loss: 58.332714, accuracy: 0.925781


In [8]:
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.911100
